<div style="font-family: 'Gen Jyuu Gothic Monospace Medium', 'Noto Sans TC', 'Inconsolata'; font-size: 440%; font-weight: 700; text-align: center; color: Plum;">
<br>
寫個無聊「遊戲」來練功
<br>
<br>
<br>
<!-- <div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 180%; text-align: center;"> -->

</div>

In [200]:
%%javascript
// 設定output文字顏色。
document.styleSheets[0].addRule('body', 'color: #87CEFA !important;')

<IPython.core.display.Javascript object>

<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
遊戲規則
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 搜集或隨機產生一個英文單字庫(目前該字庫已有37多萬字)。
* 從字庫中取出每一字檢查是否為`ShiftLex`。
* 所謂`ShiftLex`，是筆者和ChatGPT討論後創出的新詞，意思是：
    * 該字本身是有意義的英文單字。
    * 將每一個字母輪流移至(shift)字首，其餘字母順序不變。所有移動後形成的新字都必須為有效英文單字。
* `ShiftLex`範例(目前暫未找出4個字母以上的`ShiftLex`)：
    * art -> rat -> tar
    * tea -> eta -> ate    


<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
如可「隨機產生」英文單字？
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 簡單：用`random_word` package即可。
* `$ pip install random-word`


In [267]:
from random_word import RandomWords

r = RandomWords()
words = set()
for i in range(9):
    # print(f'{i}', end='\r')
    words.add(r.get_random_word())
print(f'{len(words) =}')
print(f'{words}')

len(words) =9
{'academe', 'granomerite', 'misbegun', 'modulet', 'grahams', 'howsomever', 'semideltaic', 'haloxylin', 'calcedon'}


<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
不過這樣隨機產生太慢了！
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 直接到該package的github，將它[整個字庫](https://raw.githubusercontent.com/vaibhavsingh97/random-word/master/random_word/database/words.json)copy出來比較快。
* 光用這個字庫筆者嫌不夠(筆者很「龜毛」的)，又從其他網站copy了很多單字，全部轉為小寫後存入`words_raw.dat`檔。
* 這個檔案只有最簡單的整理，尚待進一步去蕪存菁，例如過濾重複、刪除過短的字、刪除帶數字或特殊符號者...
* 整理的code在下面。

In [251]:
# 單字字庫整理-1：刪除重複及全轉為小寫
with open('./words_raw.dat', 'r') as f1:
    words = list(set(f1.read().lower().split('\n')))

# 單字字庫整理-2：最少3個字母、字母沒有完全相同、
# 且沒有數字和特殊符號者才納入最終字庫。
symbols = ('.', ',', '?', '_', '/', '\\', '(', ')', '[', ']', '{', '}', '<', '>', '"', "'", '|', ':', ';', '~', '!', '`', '@', '#', '$', '%', '^', '&', '*', '+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ' ', '=', 'þ')
with open('./words_final.dat', 'w') as f2:
    for index, word in enumerate(words):
        leng_word = len(word)
        if leng_word >= 3 and word != word[0]*leng_word and not [char for char in word if char in symbols]:
            print(f'{index}', end='\r')
            _ = f2.write(f'{word}\n')
            f2.flush()

<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
資料整理好後，就開始在資料檔逐個檢查了。
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 如何檢查
* 使用enchant模組

In [171]:
import nltk   # .corpus import words

nltk.download('words')
"Alex" in words.words()

[nltk_data] Downloading package words to /home/alex/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

True

In [236]:
from nltk.corpus import words
"Taipei" in words.words()



False

In [237]:
from spellchecker import SpellChecker
# import spellchecker

spell = SpellChecker()
def check(word):
    if word == spell.correction(word):
        return True
    else:
        return False
    
check('Taipei')
check('taipei')    

True

True

In [239]:
import enchant

spellings = ('en_', 'en_US')
for spelling in spellings:
    d = enchant.Dict(spelling)
    word1 = 'Taiwan'
    word2 = 'taiwan'
    print(f"{spelling}\t\t{word1}: {d.check(word1)}\t{word2}: {d.check(word2)}")
    # d.check('gray')
# d.check("Helo")
# d.suggest("Helo")

en_		Taiwan: True	taiwan: False
en_US		Taiwan: True	taiwan: False


* 決定應用`en_UK`(英式英文拼法字典)

* 讀取詞彙檔

In [163]:
enchant.dict_exists("e_..")
enchant.list_languages()

False

['en', 'en_AU', 'en_CA', 'en_GB', 'en_US', 'en_ZA']

In [254]:
with open('./words_final.dat', 'r') as f:
    words = f.read().split('\n')
    
words[-1]     

'zyzzyvas'

In [249]:
# v1: 直白寫  
import enchant

d = enchant.Dict('en_')
shift_lexes = []
for word in words:
    fitting = True
    for index, alphabet in enumerate(word):
        print(f'{index:,} / {len(words):,}', end='\r')
        this_rotation = f'{alphabet}{word[:index]}{word[index+1:]}' 
        if not d.check(this_rotation.lower()):
            fitting = False
            break
    if fitting:
       shift_lexes.append(word)
        
        
print(shift_lexes)
print(len(shift_lexes))

['tea', 'ups', 'alp', 'rob', 'arm', 'apt', 'int', 'add', 'asp', 'amt', 'opp', 'app', 'own', 'art', 'wee', 'eel', 'opt', '']
18


In [255]:
# v2: 改為function
import enchant
def is_valid_shift_lex(lex: str, rotations: int) -> bool:
    d = enchant.Dict('en_UK')
    fittings = 0
    for index, alphebet in enumerate(lex):
        this_rotation = f'{alphebet}{lex[:index]}{lex[index+1:]}' 
        if d.check(this_rotation):
            fittings += 1
            if fittings >= rotations:
                break
    return fittings >= rotations or fittings == len(lex)

shift_lexes = []
for index, word in enumerate(words):
    print(f'{index:,} / {len(words):,}', end='\r')
    if is_valid_shift_lex(lex=word, rotations=3):
       shift_lexes.append(word)
        
print(shift_lexes)
print(len(shift_lexes))

['abbe', 'abbes', 'acme', 'acre', 'acres', 'add', 'adds', 'aids', 'alp', 'alps', 'alts', 'amt', 'angers', 'app', 'appal', 'apps', 'apt', 'arced', 'arcing', 'arm', 'arms', 'arrest', 'art', 'arts', 'asp', 'asps', 'assess', 'avers', 'cares', 'caress', 'cats', 'door', 'doors', 'earring', 'east', 'eats', 'educes', 'eel', 'eels', 'ends', 'eviler', 'greet', 'greets', 'hanks', 'hears', 'hips', 'hist', 'hits', 'huts', 'ides', 'int', 'isms', 'kiss', 'labs', 'lams', 'laps', 'lats', 'lumps', 'manatee', 'manatees', 'mites', 'ones', 'oops', 'opp', 'opt', 'opts', 'ores', 'orts', 'own', 'parses', 'pass', 'pots', 'rapt', 'ribbed', 'ribber', 'ribbers', 'ribbing', 'rob', 'robs', 'steer', 'steers', 'tea', 'ups', 'veer', 'veneer', 'wast', 'watt', 'watts', 'wee', 'weer', 'wees', 'windless']
92


In [70]:
# v3: 改為class
import enchant

class ShiftLex():
    def __init__(self):
        self.d = enchant.Dict('en_UK')
        
    def is_valid(self, lex: str, rotations: int) -> bool:
        fitting = True
        for index, alphabet in enumerate(lex):
            this_rotation = f'{alphabet}{lex[:index]}{lex[index+1:]}' 
            if not self.d.check(this_rotation):
                fitting = False
                break
        return fitting
    

word = ShiftLex() 
shift_lexes = []
for word in words:
    if word.is_valid(lex=word, rotations=3):
       shift_lexes.append(word)
        
print(shift_lexes)
print(len(shift_lexes))

['tea', 'ups', 'alp', 'rob', 'arm', 'apt', 'int', 'add', 'asp', 'amt', 'opp', 'app', 'own', 'art', 'wee', 'eel', 'opt']
17
